Código para la creación de imagen tiff con metadatos de drone

In [27]:
from PIL import Image
from PIL.TiffTags import TAGS
import re
import rasterio
import numpy as np

In [28]:
# obtenemos la imagen .tif
imagen = "./data/fotos/DJI_0051.TIF"
img = Image.open(imagen)
# obtenemos los Tags de la metadata y se almacenan en un diccionario
meta_dict = {TAGS[key]: img.tag[key] for key in img.tag_v2}
# Se imprime el diccionario para obtener la composición de los datos
# for rec in meta_dict:
#     print(rec, ":", meta_dict[rec])


In [29]:
# Extraemos el indicador xmp
p = meta_dict.get("XMP")
s = p.decode("UTF-8")
# dividimos por el salto de linea y obtenemos una lista
div = s.split("\n")

In [30]:
#eliminamos los espacios vacios 

for ind,recorrido in enumerate(div):
    div[ind]=recorrido.strip() 

div[17]

'drone-dji:AbsoluteAltitude="+84.65"'

In [31]:
usar = div[17]

result = re.search(":(.*)=", div[17])
result.group(1)

result2 = re.search("\"(.*)\"", div[17])
result2.group(1)

print(result.group(1))
print(result2.group(1))


AbsoluteAltitude
+84.65


Creo un diccionario donde almaceno las variables de la metadata con su valor

In [32]:
metadiccionario = {}
for ind,recorrido in enumerate(div):
    try:
        metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
    except:
        pass

print(metadiccionario)

{'xmpmeta xmlns:x': 'adobe:ns:meta/', 'RDF xmlns:rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'Description rdf:about': 'DJI Meta Data', 'tiff': 'http://ns.adobe.com/tiff/1.0/', 'exif': 'http://ns.adobe.com/exif/1.0/', 'xmp': 'http://ns.adobe.com/xap/1.0/', 'xmpMM': 'http://ns.adobe.com/xap/1.0/mm/', 'dc': 'http://purl.org/dc/elements/1.1/', 'crs': 'http://ns.adobe.com/camera-raw-settings/1.0/', 'drone-dji': 'http://www.dji.com/drone-dji/1.0/', 'Camera': 'http://pix4d.com/camera/1.0', 'ModifyDate': '2022-02-11', 'CreateDate': '2022-02-11', 'Make': 'DJI', 'Model': 'FC6360', 'format': 'image/TIF', 'AbsoluteAltitude': '+84.65', 'RelativeAltitude': '+70.03', 'GpsLatitude': '11.22617803', 'GpsLongitude': '-74.18549349', 'GimbalRollDegree': '+0.00', 'GimbalYawDegree': '-96.50', 'GimbalPitchDegree': '-89.90', 'FlightRollDegree': '+6.40', 'FlightYawDegree': '-95.30', 'FlightPitchDegree': '+6.50', 'FlightXSpeed': '+0.10', 'FlightYSpeed': '+0.40', 'FlightZSpeed': '+0.00', 'CamReverse': '0

Se calcula la resolución en metros por pixel.
Es decir, a cuantos metros equivale un pixel

In [33]:
resolucion = float(metadiccionario["RelativeAltitude"])/float(metadiccionario["CalibratedFocalLength"])
resolucion

0.03660104451823564

In [34]:
# Abrir la imagen
with rasterio.open(imagen) as src:
    # Obtener la información de la imagen
    width = src.width
    height = src.height
    transform = src.transform
    crs = src.crs


c:\Users\Migue Polo\.virtualenvs\kriging-y0ZA2chP\lib\site-packages\rasterio\__init__.py:332: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Cálculo de las coordenadas de las esquinas la imagen

Se divide entre 111111 para convertir los metros en grados. Esto se debe a que 1 grado de longitud o latitud en el ecuador es aproximadamente 111.111 km de ancho. Dado que la longitud y la latitud son medidas en grados, se debe convertir el tamaño de los píxeles en metros a grados para poder calcular las coordenadas extremas en grados.
Donde el ancho de la imagen en metros se divide entre la distancia en metros por grado para obtener la diferencia en grados. El factor 111111 es una aproximación, ya que la distancia en metros por grado varía según la latitud. Sin embargo, esta aproximación es suficiente para una precisión razonable.

Es importante tener en cuenta que esta conversión solo es válida para latitudes cercanas al ecuador, ya que la distancia en metros por grado varía según la latitud.

In [35]:
# Calcular las coordenadas de los extremos en metros
min_lon = float(metadiccionario["GpsLongitude"]) - (
    float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
max_lon = float(metadiccionario["GpsLongitude"]) + (
    float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
min_lat = float(metadiccionario["GpsLatitude"]) - (
    float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
max_lat = float(metadiccionario["GpsLatitude"]) + (
    float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
print(min_lon, max_lon, min_lat, max_lat)


-74.18575701778406 -74.18522996221594 11.225963913675452 11.226392146324548


In [36]:
transform = rasterio.transform.from_bounds(
    min_lon, min_lat, max_lon, max_lat, img.width, img.height)
transform


Affine(3.2940973007100636e-07, 0.0, -74.18575701778406,
       0.0, -3.2940973007373926e-07, 11.226392146324548)

Leo la imagen con rasterio y luego la guardo con las coordenadas correspondientes y como archivo tiff

In [37]:
# Abrir el archivo original
with rasterio.open(imagen) as src:
    # Leer los datos de la imagen
    image = src.read()

    # Crear una copia del archivo
    with rasterio.open(
        'data/test_drone51x.TIF',
        'w',
        driver='GTiff',
        height=src.height,
        width=src.width,
        count=1,
        dtype="float64",
        crs={"init": "epsg:4326"},
        transform=transform,
    ) as dst:
        dst.write(image)


*Aquí empiezo a probar la rotación de la imagen*

In [19]:
from rasterio.warp import transform_bounds
from rasterio.transform import Affine
import math

In [26]:
# Abrir el archivo original
with rasterio.open('data/test_drone51.TIF') as src:
    # Leer los datos de la imagen
    image = src.read()

    # Obtener los metadatos de la imagen
    profile = src.profile
    print(profile)
    # Obtener la transformación afín del archivo
    transform = src.transform

    # Calcular la transformación afín rotada
    # angle = float(metadiccionario["GimbalYawDegree"])
    # Calcular la transformación afín rotada
    rotation = Affine.rotation(math.radians(-96.50))
    new_transform = rotation * transform

    # # Calcular las esquinas rotadas
    # ul, ur, ll, lr = transform_bounds(
    #     profile["crs"], new_transform, *src.bounds)
    # print("Esquinas rotadas:")
    # print("UL:", ul)
    # print("UR:", ur)
    # print("LL:", ll)
    # print("LR:", lr)

    # # Crear una copia del archivo
    # with rasterio.open(
    #     'data/test_drone51_rotate3.TIF',
    #     'w',
    #     driver='GTiff',
    #     height=src.height,
    #     width=src.width,
    #     count=1,
    #     dtype="float64",
    #     crs={"init": "epsg:4326"},
    #     transform=transform,
    # ) as dst:
    #     dst.write(src.read())


{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 1600, 'height': 1300, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(3.9817983221723807e-07, 0.0, -74.18581203386577,
       0.0, -3.9817983222543666e-07, 11.226436846890946), 'blockysize': 1, 'tiled': False, 'interleave': 'band'}


In [12]:
import contextily as cx

In [13]:
ax = src.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
cx.add_basemap(ax)

AttributeError: 'DatasetReader' object has no attribute 'plot'

In [ ]:
from rasterio.plot import show
import matplotlib.pyplot as plt

In [ ]:
# Abrir el archivo raster
with rasterio.open('data/test_drone51.TIF') as src:
    raster = src.read()
    transform = src.transform
    crs = src.crs

#Crear una figura
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_aspect('equal')

# Agregar el raster al mapa base
show(raster, transform=transform, ax=ax)

# Agregar las coordenadas
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title("Mapa Base con el raster")

# Mostrar la figura
plt.show()
cx.add_basemap(ax)

In [ ]:
import folium
import rasterio
from rasterio.plot import show
from folium import plugins


In [ ]:
import folium


In [ ]:
# Cargar el raster
with rasterio.open('data/test_drone51.TIF') as src:
    raster = src.read()
    transform = src.transform
    bounds = src.bounds

# Crear el mapa base
m = folium.Map(location=[(bounds.top + bounds.bottom)/2,
               (bounds.left + bounds.right)/2], zoom_start=18)

# Añadir el raster al mapa
folium.raster_layers.ImageOverlay(
    image="data/test_drone51.TIF",
    bounds=[[bounds.top, bounds.left], [
        bounds.bottom, bounds.right]],
    opacity=0.5,
    # colormap=lambda x: (1, 0, 0, x),
).add_to(m)

# Mostrar el mapa
m
